In [1]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "CompVis/stable-diffusion-v1-4"
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16 if device == "cuda" else torch.float32)
pipe = pipe.to(device)



c:\Users\deepa\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading pipeline components...: 100%|██████████| 7/7 [00:12<00:00,  1.73s/it]


In [7]:
prompt = "Young person looking happy."

with torch.autocast("cuda" if device == "cuda" else "cpu"):
    image = pipe(prompt).images[0]

image.save("reference.jpg")

print("Image saved as 'reference.jpg'")


100%|██████████| 50/50 [03:14<00:00,  3.89s/it]


Image saved as 'reference.jpg'


In [8]:
from PIL import Image
from torchvision import transforms
import torch
from skimage.metrics import structural_similarity as ssim
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((256, 256)), 
    transforms.ToTensor(),  
])

def evaluate_image(generated_image_path, reference_image_path):
    generated_image = Image.open(generated_image_path).convert("RGB")
    reference_image = Image.open(reference_image_path).convert("RGB")
    
    generated_image = transform(generated_image).unsqueeze(0).to(device)
    reference_image = transform(reference_image).unsqueeze(0).to(device)
    
    generated_image = generated_image.squeeze().permute(1, 2, 0).cpu().numpy()
    reference_image = reference_image.squeeze().permute(1, 2, 0).cpu().numpy()
    
    ssim_value = ssim(generated_image, reference_image, data_range=1.0, channel_axis=2)
    return ssim_value

ssim_value = evaluate_image('generated_image.png', 'reference.jpg')
print(f'SSIM between generated and reference image: {ssim_value:.4f}')


SSIM between generated and reference image: 0.2343
